# Feedforward Neural Network built on Keras

Below is a program built on Keras to build a regression model. 

## Dependencies

In this work, we need the numpy and pandas for processing raw data, and we will use keras (Tensorflow backend) an scikit-learn for building and evaluating regression models.

In [1]:
import numpy as np
import pandas as pd
import keras.callbacks
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

Using TensorFlow backend.


## More preprocessing

In this step, we put all the idle and wild data into one file for the purpose of training and testing a regression model later. We also get rid of timestamps because we are using the emg and x,y,z values (features) at one moment and predicting the x,y,z values (targets) at the next moment. We could add timestamps into features as well.

In [16]:
files = ['miller_idleprocessed', 'sam_wildprocessed', 'layton_idleprocessed', 'miller_wildprocessed', 'val_idleprocessed', 
         'layton_wildprocessed', 'sam_idleprocessed', 'val_wildprocessed']
frames = [pd.read_csv(f, header = None) for f in files]
dataframe = pd.concat(frames)   #Concatenate all preprocessed files into one file

In [51]:
dataset_new = dataframe.values
dataset_new = np.delete(dataset_new, [1, 5], axis = 1)  #delete the second and the sixth items (timestamps) in each array
X = dataset_new[:-1]  # X is the features set
Y = dataset_new[1:]   # Y is the target set
Y = np.delete(Y, [0], axis = 1)  # Y is the targeted set and we don't need the EMG value in Y. Only x, y, z.
print (X,Y)

[[  4.07200000e+03  -3.28000000e-01   2.26500000e-01  -9.49000000e-01]
 [  4.04300000e+03  -3.16000000e-01   2.14800000e-01  -9.37000000e-01]
 [  4.04200000e+03  -3.35000000e-01   2.26500000e-01  -9.45000000e-01]
 ..., 
 [  3.48300000e+03  -5.40000000e-02  -7.07000000e-01   6.25000000e-01]
 [  3.49500000e+03  -4.20000000e-02  -7.14000000e-01   6.17100000e-01]
 [  3.48400000e+03  -5.80000000e-02  -7.26000000e-01   6.17100000e-01]] [[-0.316   0.2148 -0.937 ]
 [-0.335   0.2265 -0.945 ]
 [-0.324   0.2382 -0.937 ]
 ..., 
 [-0.042  -0.714   0.6171]
 [-0.058  -0.726   0.6171]
 [-0.042  -0.746   0.6171]]


## Build the model by using Keras

* In this part, I benefited a lot from this post: https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/.
* I put 16 hidden layers and use relu as the activation layer. (It turns out that if I put the number of hidden layers under 10, it doesn't really work.) I use mean_squared_error as the loss function. And I use the adam optimizer. (Adam is better than RMSprop based on my experiment. In one experiment I did with RMSprop, the r2 score is only around 0.89, which is around 9 percent less than when using adam. RMSprop optimizer also learns slower.)
* The official Keras document on Sequential model can be found here: https://keras.io/getting-started/sequential-model-guide/

In [55]:
def baseline_model():
    model = Sequential()
    model.add(Dense(16, input_dim = 4, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(3, kernel_initializer = 'normal'))
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    return model

## Testing performance of the model

* A large part of the code here is from Layton. It uses the modules in scikit-learn. The normal scoring of my model, I believe, is mean sqaured error. I changed into r2 in order to have better comparison metrics. It we get rid of  *scoring = 'r2'*, then the printed resulted reg.score will be mean squared error. 
* Here are a couple helpful links from scikit-learn documents on how to set parameters for cross validation here. 
    - http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    - http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [60]:

# from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33)

from sklearn.model_selection import GridSearchCV

#hyperparameters to search through with CV
parameters = {
    'epochs' : [400],
    'batch_size' : [16,32], # The larger the batch size, the faster the model will run.
    'verbose' : [2] # for debugging purpose. It will significantly lower the running speed. To speed up, change it to 0.
}

#searching through all combinations of parameters to find best (ranked by CV)
reg = GridSearchCV(KerasRegressor(build_fn=baseline_model), parameters, scoring = 'r2')
reg.fit(X_train, y_train)

#scoring against test set to see how good our model is
print(reg.score(X_test, y_test))

Epoch 1/400
 - 10s - loss: 1.7724
Epoch 2/400
 - 7s - loss: 0.1631
Epoch 3/400
 - 6s - loss: 0.1082
Epoch 4/400
 - 6s - loss: 0.0986
Epoch 5/400
 - 6s - loss: 0.0424
Epoch 6/400
 - 6s - loss: 0.0584
Epoch 7/400
 - 7s - loss: 0.0285
Epoch 8/400
 - 8s - loss: 0.0218
Epoch 9/400
 - 7s - loss: 0.0185
Epoch 10/400
 - 7s - loss: 0.0216
Epoch 11/400
 - 6s - loss: 0.0202
Epoch 12/400
 - 6s - loss: 0.0178
Epoch 13/400
 - 6s - loss: 0.0148
Epoch 14/400
 - 6s - loss: 0.0167
Epoch 15/400
 - 6s - loss: 0.0184
Epoch 16/400
 - 7s - loss: 0.0175
Epoch 17/400
 - 6s - loss: 0.0146
Epoch 18/400
 - 7s - loss: 0.0173
Epoch 19/400
 - 6s - loss: 0.0131
Epoch 20/400
 - 6s - loss: 0.0129
Epoch 21/400
 - 7s - loss: 0.0142
Epoch 22/400
 - 6s - loss: 0.0135
Epoch 23/400
 - 6s - loss: 0.0135
Epoch 24/400
 - 7s - loss: 0.0151
Epoch 25/400
 - 8s - loss: 0.0132
Epoch 26/400
 - 10s - loss: 0.0135
Epoch 27/400
 - 11s - loss: 0.0130
Epoch 28/400
 - 13s - loss: 0.0143
Epoch 29/400
 - 7s - loss: 0.0115
Epoch 30/400
 - 7s 

## Let's print some results and see the performance!

Code here is from Layton too.

In [59]:
print('data:')
print(X_test[:10])
print('\npredictions:')
print(reg.predict(X_test)[:10])
print('\ntargets:')
print(y_test[:10])

data:
[[  8.01700000e+03   1.17000000e-02  -4.14000000e-01   7.18700000e-01]
 [  4.81100000e+03  -1.32000000e-01   3.67100000e-01  -9.37000000e-01]
 [  3.31500000e+03  -3.08000000e-01   5.00000000e-01  -9.02000000e-01]
 [  1.20310000e+04   3.90000000e-03  -5.62000000e-01   7.46000000e-01]
 [  3.06400000e+03   0.00000000e+00  -5.78000000e-01   7.22600000e-01]
 [  6.89800000e+03   5.66400000e-01   7.03000000e-02  -5.03000000e-01]
 [  3.50800000e+03   3.16400000e-01   3.04600000e-01  -8.43000000e-01]
 [  7.46300000e+03   9.02300000e-01  -1.36000000e-01   2.69500000e-01]
 [  3.17700000e+03  -7.00000000e-03   5.42900000e-01  -8.94000000e-01]
 [  3.06300000e+03  -2.30000000e-02  -5.78000000e-01   7.18700000e-01]]

predictions:
[[-0.04756569 -0.45818153  0.70033002]
 [-0.16557302  0.34305984 -0.95755291]
 [-0.32143134  0.47893479 -0.91633141]
 [-0.08448749 -0.61465156  0.70999932]
 [-0.00862433 -0.60292488  0.72980273]
 [ 0.47141322  0.02984963 -0.52872503]
 [ 0.25670877  0.28976241 -0.823730